以下のノートブックを実行するには、まだ設定していない場合、.envファイル内に `OPENAI_API_KEY` としてOpenAIキーを設定する必要があります。


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

次に、Embedding IndexをPandasのDataframeに読み込みます。Embedding Indexは`embedding_index_3m.json`というJSONファイルに保存されています。Embedding Indexには、2023年10月末までの各YouTubeトランスクリプトの埋め込みが含まれています。


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

次に、クエリに対して埋め込みインデックスを検索する `get_videos` という関数を作成します。この関数は、クエリに最も類似した上位5つの動画を返します。関数の動作は以下の通りです。

1. まず、埋め込みインデックスのコピーを作成します。
2. 次に、OpenAIの埋め込みAPIを使用してクエリの埋め込みを計算します。
3. その後、埋め込みインデックスに `similarity` という新しい列を作成します。`similarity` 列には、クエリの埋め込みと各動画セグメントの埋め込みとのコサイン類似度が含まれます。
4. 次に、`similarity` 列で埋め込みインデックスをフィルタリングします。コサイン類似度が0.75以上の動画のみを含むようにフィルタリングします。
5. 最後に、`similarity` 列で埋め込みインデックスをソートし、上位5つの動画を返します。


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

この関数は非常にシンプルで、検索クエリの結果を出力するだけです。


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. まず、Embedding IndexがPandasのDataframeに読み込まれます。  
2. 次に、ユーザーにクエリの入力を促します。  
3. その後、`get_videos`関数が呼び出され、Embedding Index内でクエリを検索します。  
4. 最後に、`display_results`関数が呼び出され、結果をユーザーに表示します。  
5. その後、ユーザーに再度クエリの入力を促します。このプロセスはユーザーが`exit`と入力するまで続きます。  

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ja.png)

クエリの入力を求められます。クエリを入力してEnterキーを押してください。アプリケーションはクエリに関連するビデオのリストを返します。また、質問の答えがビデオのどの部分にあるかへのリンクも返します。

試してみるクエリの例は以下の通りです：

- Azure Machine Learningとは何ですか？  
- 畳み込みニューラルネットワークはどのように動作しますか？  
- ニューラルネットワークとは何ですか？  
- Azure Machine LearningでJupyter Notebooksを使えますか？  
- ONNXとは何ですか？


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：  
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されました。正確性の向上に努めておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。原文の言語による文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じた誤解や誤訳について、当方は一切の責任を負いかねます。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
